In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


#plt.imshow(train_images[0], cmap="Greys_r")

#process traindata
train_images = (train_images.astype(np.float32) / 255.).reshape((-1, 784))
train_labels = train_labels.astype(np.int32)
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.shuffle(1000)
train_data = train_data.repeat()
train_data = train_data.batch(128)
iterator = iter(train_data)


#process textdata
test_images = (test_images.astype(np.float32) / 255.).reshape((-1, 784))
test_labels = test_labels.astype(np.int32)


train_steps = 2000
learning_rate = 0.1


W1 = tf.Variable(tf.compat.v1.random.normal([784,500], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1,shape=[500]))

W2 = tf.Variable(np.zeros([500, 10]).astype(np.float32))
b2 = tf.Variable(np.zeros(10, dtype=np.float32))




for step in range(train_steps):
    img_batch, lbl_batch = next(iterator)
    with tf.GradientTape() as tape:
        logits = tf.matmul(tf.nn.relu(tf.matmul(img_batch, W1) + b1),W2)+b2
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
           logits=logits, labels=lbl_batch))
    grads = tape.gradient(xent, [W1,b1,W2, b2])
    W1.assign_sub(learning_rate * grads[0])
    b1.assign_sub(learning_rate * grads[1])
    W2.assign_sub(learning_rate * grads[2])
    b2.assign_sub(learning_rate * grads[3])
    
    if not step % 200:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))

        
test_preds = tf.argmax(tf.matmul(tf.nn.relu(tf.matmul(test_images, W1) + b1),W2)+b2, axis=1,
                       output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels),
                             tf.float32))
print(acc)

#1. shuffle/batch/repeat: the data is first shuffled, and then divided into batches, and finally repeated. 

#2.shuffle/repeat/batch:The difference between "1." and "2." is that if the samples in an epoch cant be divided
# by the size of the batch, "1." will leave a small "tail" in each epoch, and 2. will leave only one epoch 
#at the end of epochs

#3.batch/shuffle/repeat : .shuffle after .batch, it will shuffle the batch order , but it will not schuffle the 
#data in each batch.

#4.batch/repeat/shuffle : only batch order will be shuffled, the data in each batch will not be shuffled,and 
#the batches between epoch will be shuffled (when some batches appear twice, other batches heve not appeared yet),

#5.repeat/shuffle/batch/ : the data between epoch will be shuffled (when some data appears twice, other data has 
#not appeared yet)

#6.repeat/batch/shuffle：only the oder of the batchs will be shuffled.The difference between "3." and "6." is that 
#if the samples in an epoch cant be divided by the size of the batch, "2." will leave a small "tail" in each epoch, 
#and 5. will leave only one epoch at the end of epochs

#i think "shuffle/repeat/batch" is the most sensible for training neural networks, the data in each epoch will be shuffled,
#and there is only one "tail" at the end of epochs, if the samples in an epoch cant be divided by the size of the batch.



Loss: 2.3025853633880615 Accuracy: 0.140625
Loss: 0.35939326882362366 Accuracy: 0.890625
Loss: 0.31237682700157166 Accuracy: 0.9296875
Loss: 0.25450196862220764 Accuracy: 0.9375
Loss: 0.33655261993408203 Accuracy: 0.890625
Loss: 0.2841366231441498 Accuracy: 0.9140625
Loss: 0.18425258994102478 Accuracy: 0.9453125
Loss: 0.13215544819831848 Accuracy: 0.96875
Loss: 0.14936870336532593 Accuracy: 0.9609375
Loss: 0.2092491090297699 Accuracy: 0.9296875
tf.Tensor(0.9565, shape=(), dtype=float32)
